#Computing the volume underneath a Pareto frontier
This program computes the volume under a frontier of Pareto-optimal solutions to a multi-criterion optimization model. This volume provides a measure of the conflict among the objectives.

In [60]:
import pandas as pd
import numpy as np

### Preparation for the volume computation algorithm

In [221]:
# Get the solutions that define the pareto frontier.
# It is assumed that these frontiers do not contain any dominated solutions
# (that is, that the solutions are truly Pareto optimal)

sols = pd.read_csv("solutionSets/3d/OptimalSolutions_2014XchgRate.csv")

In [222]:
# Hard-coded manipulations to tidy-up sols

sols = sols.drop(["SolutionIndex"],axis=1) #remove solution index column

In [223]:
# Get list of objectives from the dataframe

objs = sols.columns.tolist()

In [224]:
# Create empty dictionary to hold the objectives and the senses of each
# This process is hard-coded and requires some knowledge about the model
# that resulted in the sols dataframe

objSenses = {}

In [225]:
# Hard-coded manipulations to properly set objSenses (max/min)
# 1 for max, and 0 for min

for obj in objs:
    objSenses[obj] = 1 # all objectives are max for curr example
#objSenses[" Risk"] = 0

In [227]:
# Get the objectives' bounds

objBounds = {}
for obj,sense in objSenses.iteritems():
    objBounds[obj] = [sols[obj].min(),sols[obj].max()]

In [228]:
# Get each objective's ideal value

idealObjVals = {}
for obj,bounds in objBounds.iteritems():
    idealObjVals[obj] = bounds[objSenses[obj]] # 0th entry of bounds is min, 1st is max

In [229]:
# Get each objective's worst value

worstObjVals = {}
for obj,bounds in objBounds.iteritems():
    worstObjVals[obj] = bounds[not(objSenses[obj])]

In [230]:
# Compute the non-normalized volume of the objective space

objSpaceVolume_nonNorm = 1
for obj,bounds in objBounds.iteritems():
    objSpaceVolume_nonNorm *= bounds[1] - bounds[0]

In [231]:
# Normalize the objective space by converting each value to the relative achievement along its axis:

#   distance from the worst case value
# ---------------------------------------
# total distance spanned by the objective

# Objective space is now in [0,1]^N (N is number of objectives)


for obj,bounds in objBounds.iteritems():
    sols[obj] = sols[obj].apply(
        lambda x: \
        abs(x - worstObjVals[obj]) / \
        (bounds[1]-bounds[0]))

### Algorithm to compute volume of frontier

#### Algorithm 1: "Add-the-sides" method
(Sandor)

In [232]:
# V = sum_{i \in N} A_i z_i
# where
# A_0 = 0
# and
# A_i = x_i y_i
    # - A_{i-1}
    # + \sum_{j<i : x_j > x_i} y_j \left[ x_j - \max_{l \le i}\{x_l : x_l < x_j \} \right]
    # + \sum_{j<i : y_j > y_i} x_j \left[ y_j - \max_{k \le i}\{y_k : y_k < y_j \} \right]
    
# Have not yet generalized to N dimensions

#### Algorithm 2: "Subtract-the-middle" method
(Nick) 

In [233]:
# Initialize frontier volume

frontierVolume = 0

In [234]:
# Initialize the array of solutions' (N-1)-dimensional volume contributions

volContributions = {}

In [235]:
# Old method sketch:
# V = sum_{i \in N} A_i z_i
# where
# A_0 = 0
# and
# A_i = x_i y_i
    # - \sum_{s \in S_*} \min(x_*,x_s) \Delta_{y_s,y_{s-1}}
    # - x_{N^*} \left( y_* - \max_s\{ y_s : s \in S_* \} \right)

In [236]:
# Sort the dataframe in descending order from first column

sols = sols.sort_values(by=[objs[0]],ascending=False)

In [238]:
# Get list of all non-primary dimensions

dims_secondary = [col for col in sols.columns if col not in [objs[0]]]
dims_secondary

[' ECT', ' FV']

In [253]:
# Compute the volume

# For each solution...
for idx,row in sols.iterrows():
    
    # get its N-1 dimensional contribution, currently including overlap
    contribution = row[dims_secondary].product()
    
    # Subtract away the contributions from solutions whose (N-1)-dimensional volumes are completely enclosed
    # within the (N-1)-dimensional space created by the current solution
    
    # Get the solutions whose N-1 dimensional volumes are completely contained within this solution's N-1 dimensional space
    enclosedSols = {k: v for k, v in volContributions.iteritems() if (sols[dims_secondary].ix[k] < row[dims_secondary]).all()}
    # Subtract away the contributions from each of those solutions
    for soln,contrib in enclosedSols.iteritems():
        contribution -= contrib
        
    # Subtract away the trimmings from the nearest neighbors in each secondary dimension

In [ ]:
sols